In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

# 1950~2067년
dfDeathWoman = pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
dfDeathMan = pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
npDeathWoman = np.array(dfDeathWoman)
npDeathMan = np.array(dfDeathMan)

# 1950~2047년
dfDoctorData = pd.read_excel('../../data/doctorData.xlsx').set_index(['년도']).iloc[:98,:]
npPassDoctor = np.around(np.array(dfDoctorData[['의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]))
npDoctorExam = np.around(np.array(dfDoctorData[['의사고시 합격자수','의사고시 불합격자수']]))
npPopulation = np.array(dfDoctorData['추계인구'])
npPassDoctorRate = np.array(dfDoctorData[['의대졸비율/남', '의대졸비율/여', '의전졸비율/남', '의전졸비율/여', '불합비율/남', '불합비율/여']])
npCurrentDoctor = np.array(dfDoctorData[['의사수/계','의사수/남','의사수/여']])

# 2011~2016년
dfDoctorAgeData = pd.read_excel('../../data/의사연령별분포.xlsx')
npDoctorAgeData = np.array(dfDoctorAgeData)[:,2:]

dfFirstDoctor = pd.read_excel('../../data/firstDoctor.xlsx')
npFirstDoctor = np.array(dfFirstDoctor)[:,1:-1]

# 1950~2020년 연령분포
dfKoreaAgePopRateData = pd.read_excel('../../data/koreaAgePopRateData.xlsx')
npKoreaAgePopRateDataMan = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='MALE'])[:,2:]
npKoreaAgePopRateDataWoman = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='FEMALE'])[:,2:]

In [168]:
def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

def makeAlivePerson(npData,year):
    
    deathRate = np.array([npDeathMan[year], npDeathWoman[year]])
    deadPerson = np.around((npData*deathRate))
    resultData = npData - deadPerson
    return [resultData, deadPerson]

def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    retirePerson = np.around(npData*valueList)
    result = npData - retirePerson
    
    return [result, retirePerson]

def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

def makeNewPerson(npData, tuningSet):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열 98 x 6 x 100
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열 98 x 2 x 100
    newPersonArray = np.zeros((yearSize,2,oldSize))

    modelAry = makeArrayUseModel(tuningSet)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    
    return resultData

def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(npData, tuningSet):
    #누적인원 배열
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    
    # 98 x 2 x 100 
    resultPersonArray = np.zeros(sizeArray)
    deadPersonArray = np.zeros(sizeArray)
    retirePersonArray = np.zeros(sizeArray)
    
    resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
    resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))
    
    
    
    
    for i in range(2,yearSize):
        # 1살 올리기 / shiftData -> 2 x 100
        shiftData = shiftOld(resultPersonArray[i-1])
        # 사망률 적용 / aliveData -> 2 x 100
        aliveData = makeAlivePerson(shiftData, i)
        if i==20:
            print(aliveData)
        deadPersonArray[i] = aliveData[1]
        # 은퇴율 적용 / workData -> 2 x 100
        workData = makeWorkPerson(aliveData[0], tuningSet)
        retirePersonArray[i] = workData[1]
        # 최종 계산
        resultPersonArray[i] =  workData[0] + newPersonArray[i]

            
            
    return [resultPersonArray,newPersonArray,deadPersonArray,retirePersonArray]


def sumPeopleUseAge(npData):
    yearSize = len(npData)
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData



def calculateCost(npData):   
    npSumData = np.concatenate((npData[0][:2],npData[1][:2],npData[2][:2],npData[3][:2],npData[4][:2],npData[5][:2]),axis=0)
    npResultData = np.abs(npDoctorAgeData - npSumData)
    return npResultData


def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

def makeDataFrame(npData):
    dfList = []
    startYear = 1950
    for i in range(4):
        for j in range(2):
            dfResult = pd.DataFrame(npData[i][0][j]).T
            indexList = []
            indexList.append(startYear)

            for k in range(1,98):
                dfResult = pd.concat([dfResult,pd.DataFrame(npData[i][k][j]).T])
                indexList.append(k+startYear)

            dfResult.index = indexList 
            dfList.append(dfResult)
            
    return dfList

def makeThousandPerDoctor(dfResultPerson, npPopulation):
    startYear = 1950
    dfResultPersonSum = dfResultPerson[2].sum(axis=1)
    npResultPersonSum = np.array(dfResultPersonSum)
    dfThousandPerDoctor = pd.DataFrame(npResultPersonSum/npPopulation*1000)
    dfThousandPerDoctor.index = range(startYear, 2048)
    return dfThousandPerDoctor

def makeFuturePerson(npBasicPopulation):
    yearSize = 98
    optSize = len(npBasicPopulation)
    npFuturePerson = np.zeros([yearSize,6])
    
    for i in range(optSize):
        npDoctorExam[i+76][0] = (npBasicPopulation[i][0]*0.94)+(npBasicPopulation[i][1]*0.94) #의대 합격자
        npDoctorExam[i+76][1] = (npBasicPopulation[i][0]*0.06)+(npBasicPopulation[i][1]*0.06) # 의대 불합격자 
        
        popSum = npBasicPopulation[i][0]+npBasicPopulation[i][1]+npDoctorExam[i+75][1]

        npPassDoctorRate[i+76][0] = (npBasicPopulation[i][0]*npBasicPopulation[i][2])/popSum #의대남자비율
        npPassDoctorRate[i+76][1] = (npBasicPopulation[i][0]*(1-npBasicPopulation[i][2]))/popSum #의대여자비율
        npPassDoctorRate[i+76][2] = (npBasicPopulation[i][1]*npBasicPopulation[i][3])/popSum #의전원남자비율
        npPassDoctorRate[i+76][3] = (npBasicPopulation[i][1]*(1-npBasicPopulation[i][3]))/popSum #의전원여자비율
        npPassDoctorRate[i+76][4] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][0]+npPassDoctorRate[i+76][2])/popSum # 불합격남자비율
        npPassDoctorRate[i+76][5] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][1]+npPassDoctorRate[i+76][3])/popSum # 불합격여자비율
        
        npFuturePerson[i+76][0] = np.around(popSum * npPassDoctorRate[i+76][0])
        npFuturePerson[i+76][1] = np.around(popSum * npPassDoctorRate[i+76][1])
        npFuturePerson[i+76][2] = np.around(popSum * npPassDoctorRate[i+76][2])
        npFuturePerson[i+76][3] = np.around(popSum * npPassDoctorRate[i+76][3])
        npFuturePerson[i+76][4] = np.around(popSum * npPassDoctorRate[i+76][4])
        npFuturePerson[i+76][5] = np.around(popSum * npPassDoctorRate[i+76][5])
        
    return npFuturePerson





In [164]:
def makeDoctorData(npData, tuningSet):
    npFuturePerson = makeFuturePerson(npData)
    npNewPassDoctor = npPassDoctor + npFuturePerson
    newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])
    resultData = makeResultPersonArray(newPerson,tuningSet[1])

    dfData = makeDataFrame(resultData)

    dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
    dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
    dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
    dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
    dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, npPopulation) # 1000명당 의사수
    
    return [dfResultPerson, dfNewPerson, dfDeadPerson, dfRetirePerson, dfThousandPerDoctor]

In [ ]:
# 77년 이전 신규의사수 추정
# 98 x 2 x 100 
resultPersonArray = np.zeros([27,2,100])
deadPersonArray = np.zeros([27,2,100])
retirePersonArray = np.zeros([27,2,100])
inArmy = np.zeros([27,2])
outArmy = np.zeros([27,2])
deadPerson = np.zeros([27,2])
retirePerson = np.zeros([27,2])

# 은퇴율 튜닝값
tuningSetRetireRate = [[1.2, 1.1],[30, 30],[1.8, 0.1]]

# 초기인원 입력
resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))

yearSize = 27

for i in range(2,yearSize):
    # 1살 올리기 / shiftData -> 2 x 100
    shiftData = shiftOld(resultPersonArray[i-1])
    # 사망률 적용 / aliveData -> 2 x 100
    aliveData = makeAlivePerson(shiftData, i)
    deadPersonArray[i] = aliveData[1]
    # 은퇴율 적용 / workData -> 2 x 100
    workData = makeWorkPerson(aliveData[0], tuningSetRetireRate)
    retirePersonArray[i] = workData[1]
    # 최종 계산
    resultPersonArray[i] =  workData[0] + newPersonArray[i]
    


In [169]:
#의대수, 의전원수, 의대남비율, 의전원남비율 
npBasicPopulation = np.zeros([22,4])
for i in range(22):
    npBasicPopulation[i] = np.array([3400,50,0.6,0.6])
    
tuningSetAgeRate = [[0.05, 0.05, 0.05, 0.05, 0.6, 0.1],[25,25,29,29,26,26],[40, 40, 40, 40, 40, 40]]
tuningSetRetireRate = [[1.2, 1.1],[30, 30],[1.8, 0.1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]

npFuturePerson = makeFuturePerson(npBasicPopulation)
npNewPassDoctor = npPassDoctor + npFuturePerson
newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])
resultData = makeResultPersonArray(newPerson,tuningSet[1])

dfData = makeDataFrame(resultData)

dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, npPopulation) # 1000명당 의사수
    
npSumData = sumPeopleUseAge(resultData[0][61:67])
npSumData2 = sumPeopleUseAge(resultData[0])
npCalData = calculateCost(npSumData)

npData:
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 515. 591.
  610. 564. 646. 707. 765. 549. 481. 426. 448. 260. 367. 252. 192.  91.
  196.  91. 167. 160. 158. 156. 154. 138. 136. 135. 133. 131. 116. 114.
  112. 110. 108. 102. 101.  97.  94.  91.  65.  62.  58.  53.  49.  40.
   38.  33.  29.  24.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  73.  63.
  103. 111. 117. 117.  16. 116. 209. 152. 316.  82.  74.  69.  51.  26.
   55.  28.  22.  21.  21.  21.  21.  17.  17.  17.  17.  17.  15.  15.
   15.  15.  15.  14.  14.  14.  14.  14.  11.  11.  11.  11.  11.  10.
   10.   9.   8.   7.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.  

In [156]:
dfDeadPerson[2].sum(axis=1)

1950       0.0
1951       0.0
1952      69.0
1953      74.0
1954      78.0
1955      51.0
1956      54.0
1957      61.0
1958      70.0
1959      77.0
1960      73.0
1961      78.0
1962      89.0
1963      99.0
1964     109.0
1965     108.0
1966     115.0
1967     125.0
1968     134.0
1969     140.0
1970       0.0
1971      70.0
1972      77.0
1973      82.0
1974      86.0
1975      92.0
1976      95.0
1977     182.0
1978     189.0
1979     198.0
1980     202.0
1981     207.0
1982     215.0
1983     221.0
1984     223.0
1985     227.0
1986     230.0
1987     237.0
1988     240.0
1989     247.0
1990     252.0
1991     251.0
1992     259.0
1993     272.0
1994     275.0
1995     283.0
1996     288.0
1997     296.0
1998     297.0
1999     302.0
2000     311.0
2001     308.0
2002     310.0
2003     325.0
2004     325.0
2005     327.0
2006     332.0
2007     344.0
2008     355.0
2009     365.0
2010     385.0
2011     397.0
2012     415.0
2013     418.0
2014     423.0
2015     438.0
2016     4

In [144]:
pd.DataFrame(dfResultPerson[0])

0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
1950  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1951  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1952  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1953  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1954  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1955  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1956  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1957  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1958  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1959  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1960  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1961  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1962  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1963  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1964  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1965  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1966  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1967  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1968  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1969  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1970  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1971  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1972  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1973  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1974  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1975  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1976  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1977  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1978  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1979  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1980  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1981  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1982  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1983  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1984  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1985  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1986  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1987  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1988  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1989  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1990  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1991  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1992  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1993  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1994  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1995  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1996  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1997  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1998  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1999  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2000  0.0  0.0  0.0  0.0  0.0

In [14]:
pd.DataFrame(dfRetirePerson[2].sum(axis=1))

0
1950     0.0
1951     0.0
1952     0.0
1953     0.0
1954     0.0
1955     0.0
1956     0.0
1957     0.0
1958     0.0
1959     0.0
1960     0.0
1961     0.0
1962     1.0
1963     2.0
1964     3.0
1965     4.0
1966     4.0
1967     6.0
1968     7.0
1969     8.0
1970    12.0
1971    13.0
1972    14.0
1973    15.0
1974    19.0
1975    21.0
1976    25.0
1977    30.0
1978    34.0
1979    34.0
1980    38.0
1981    39.0
1982    39.0
1983    39.0
1984    43.0
1985    44.0
1986    45.0
1987    48.0
1988    51.0
1989    55.0
1990    57.0
1991    58.0
1992    60.0
1993    65.0
1994    71.0
1995    77.0
1996    84.0
1997    87.0
1998    94.0
1999    97.0
2000   104.0
2001   111.0
2002   122.0
2003   135.0
2004   142.0
2005   156.0
2006   171.0
2007   193.0
2008   206.0
2009   225.0
2010   247.0
2011   261.0
2012   287.0
2013   308.0
2014   325.0
2015   345.0
2016   368.0
2017   398.0
2018   422.0
2019   438.0
2020   463.0
2021   483.0
2022   515.0
2023   548.0
2024   577.0
2025   617.0
2026   666.0
2027   713.0
2028   770.0
2029   821.0
2030   883.0
2031   946.0
2032  1009.0
2033  1085.0
2034  1154.0
2035  1230.0
2036  1313.0
2037  1394.0
2038  1476.0
2039  1567.0
2040  1655.0
2041  1746.0
2042  1837.0
2043  1913.0
2044  1996.0
2045  2067.0
2046  2128.0
2047  2180.0